## Used car price predictions

The objective of this project is to build a Machine Learning model which is able to predict the price of a used car in the UK.

### Import libraries

In [1]:
import sys

print(f"Python's version: {sys.version}")

Python's version: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [2]:
# General libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns

# ML libraries
import joblib
from sklearn import set_config
import category_encoders as ce 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

set_config(display='diagram')

### Ingest

Source of the dataset: [Kaggle - 100.000 UK Used cars](https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes)

In [3]:
# Check list of files
path_data = "./data"
path_data_raw = os.path.join(path_data, "raw")
path_data_interim = os.path.join(path_data, "interim")
path_data_processed = os.path.join(path_data, "processed")

print(os.listdir(path_data_raw))

FileNotFoundError: [WinError 3] The system cannot find the path specified: './data\\raw'

In [ ]:
# We want to import every .csv files except the old versions
# To be ignored: 'cclass.csv', 'focus.csv', 'unclean cclass.csv' & 'unclean focus.csv'
ignored_files = ['cclass.csv', 'focus.csv', 'unclean cclass.csv', 'unclean focus.csv']

brands = {}

for file in os.listdir(path_data_raw):
    if not file in ignored_files:
        brands[file[:-4]] = pd.read_csv(os.path.join(path_data_raw, file))

print(brands.keys())

In [ ]:
# Check if all databases have same columns
for brand, brand_data in brands.items():
    print(brand)
    print(brand_data.columns, end="\n\n")

In [ ]:
# Merge all datasets
brands_data = []

for brand, brand_data in brands.items():
    brand_data['manufacter'] = brand

    # Homogenize 'tax' column for Hyundai
    if brand == 'hyundi':
        brand_data.rename(columns={'tax(£)': 'tax'}, inplace=True)

    brands_data.append(brand_data)

used_vehicles = pd.concat(brands_data, ignore_index=True)
used_vehicles.head()

In [ ]:
used_vehicles.shape

In [ ]:
# Find N/A values
used_vehicles.info(memory_usage=True, show_counts=True)

In [ ]:
used_vehicles.to_csv(os.path.join(path_data_interim, 'used_vehicles.csv'), index=False)

In [5]:
used_vehicles = pd.read_csv('./data/used_vehicles.csv')
used_vehicles.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacter
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4,audi
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0,audi
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4,audi
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0,audi
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0,audi


### Clean

In [6]:
# Correct manufacter value
used_vehicles['manufacter'].replace({'audi': 'Audi', 'bmw': 'BMW', 'ford': 'Ford', 'hyundi': 'Hyundai', 'merc': 'Mercedes', 
                                     'skoda': 'Skoda', 'toyota': 'Toyota', 'vauxhall': 'Vauxhall', 'vw': 'Volkswagen'}, 
                                     inplace=True)

# Strip model of the vehicle
used_vehicles['model'] = used_vehicles['model'].str.strip()
used_vehicles.head()


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacter
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4,Audi
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0,Audi
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4,Audi
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0,Audi
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0,Audi


In [ ]:
difference_percent = 0.2
year_row_pb = used_vehicles.loc[used_vehicles.year == 2060]

# Find similar car
similar_car_pd = used_vehicles.loc[used_vehicles.model == year_row_pb.model.values[0]]
similar_car_pd = similar_car_pd.loc[(similar_car_pd.price >= year_row_pb.price.values[0] * (1 - difference_percent)) &
                                    (similar_car_pd.price <= year_row_pb.price.values[0] * (1 + difference_percent))]
similar_car_pd = similar_car_pd.loc[similar_car_pd.transmission == year_row_pb.transmission.values[0]]
similar_car_pd = similar_car_pd.loc[(similar_car_pd.mileage >= year_row_pb.mileage.values[0] * (1 - difference_percent)) &
                                    (similar_car_pd.mileage <= year_row_pb.mileage.values[0] * (1 + difference_percent))]
similar_car_pd = similar_car_pd.loc[similar_car_pd.fuelType == year_row_pb.fuelType.values[0]]
similar_car_pd = similar_car_pd.loc[(similar_car_pd.tax >= year_row_pb.tax.values[0] * (1 - difference_percent)) &
                                    (similar_car_pd.tax <= year_row_pb.tax.values[0] * (1 + difference_percent))]
similar_car_pd = similar_car_pd.loc[(similar_car_pd.mpg >= year_row_pb.mpg.values[0] * (1 - difference_percent)) &
                                    (similar_car_pd.mpg <= year_row_pb.mpg.values[0] * (1 + difference_percent))]
similar_car_pd = similar_car_pd.loc[similar_car_pd.engineSize == year_row_pb.engineSize.values[0]]                    

similar_car_pd


In [7]:
# Based on the previous table, the year 2060 is replaced by 2010
used_vehicles.loc[used_vehicles.year == 2060, 'year'] = 2010
used_vehicles.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacter
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4,Audi
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0,Audi
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4,Audi
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0,Audi
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0,Audi


### EDA

What is the impact of the manufacter/model to the price ?

In [ ]:
used_vehicles.hist(column='price', by='manufacter', figsize=(25,15))

What is the impact of the year of the price ?

In [ ]:
g = sns.FacetGrid(used_vehicles, col='manufacter', col_wrap=3, hue='manufacter')
g.fig.suptitle('Used cars price/year by manufacter', fontsize=15, y=1.05)
g.set(ylim=(0, used_vehicles.price.max()))
g.map(sns.regplot, 'year', 'price')

What is the impact of the transmission ?

In [ ]:
g = sns.FacetGrid(used_vehicles, col='manufacter', height=4, col_wrap=3, hue='manufacter')
g.fig.suptitle('Used cars price/transmission by manufacter', fontsize=15, y=1.05)
g.set(ylim=(0, used_vehicles.price.max()))
g.map(sns.barplot, 'transmission', 'price')

What is the impact of the mileage on the price ?

In [ ]:
g = sns.FacetGrid(used_vehicles, col='manufacter', col_wrap=3, hue='manufacter')
g.fig.suptitle('Used cars price/mileage by manufacter', fontsize=15, y=1.05)
g.set(ylim=(0, used_vehicles.price.max()))
g.map(sns.regplot, 'mileage', 'price')

What is the impact of the fuel on the price ?

In [ ]:
g = sns.FacetGrid(used_vehicles, col='manufacter', height=3.5, col_wrap=3, hue='manufacter')
g.fig.suptitle('Used cars price/fuel by manufacter', fontsize=15, y=1.05)
g.set(ylim=(0, used_vehicles.price.max()))
g.map(sns.barplot, 'fuelType', 'price')

### Preprocessing

Due to high cardinality of the features 'model' & 'manufacter', the following encoding methods have been tried:
* Binary encoder;
* LeaveOneOut encoder;
* Hashing encoder.

The other caterogical features, i.e. 'fuelType' and 'transmission', are encoded using OneHotEncoder as their dimensionality is smaller.

In [8]:
print(f"Cardinality model: {used_vehicles.model.nunique()}")
print(f"Cardinality manufacter: {used_vehicles.manufacter.nunique()}")

Cardinality model: 195
Cardinality manufacter: 9


In [9]:
# Transform year column to registration age
used_vehicles['registrationAge'] = 2021 - used_vehicles['year']
used_vehicles.drop(columns=['year'], inplace=True)
used_vehicles.head()

,model,price,transmission,mileage,fuelType,tax,mpg,engineSize,manufacter,registrationAge
0,A1,12500,Manual,15735,Petrol,150,55.4,1.4,Audi,4
1,A6,16500,Automatic,36203,Diesel,20,64.2,2.0,Audi,5
2,A1,11000,Manual,29946,Petrol,30,55.4,1.4,Audi,5
3,A4,16800,Automatic,25952,Diesel,145,67.3,2.0,Audi,4
4,A3,17300,Manual,1998,Petrol,145,49.6,1.0,Audi,2


In [10]:
# Define column transformers
# Columns model, year, price, transmission, mileage, fuelType, tax, mpg, engineSize, manufacter
numeric_features = ['mileage', 'tax', 'mpg', 'engineSize', 'registrationAge']
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_features_ohe = ['transmission', 'fuelType']
categorical_transformer_ohe = OneHotEncoder(handle_unknown='error')

categorical_features_others = ['manufacter', 'model']
categorical_transformer_bin = ce.BinaryEncoder()
categorical_transformer_loo = ce.LeaveOneOutEncoder()

# Preprocessor with BinEncoder
preprocessor_bin = ColumnTransformer(transformers=[
    ('numerical', numeric_transformer, numeric_features),
    ('categorical_low', categorical_transformer_ohe, categorical_features_ohe),
    ('categorical_high', categorical_transformer_bin, categorical_features_others)
])

# Preprocessor with LeaveOneOutEncoder
preprocessor_loo = ColumnTransformer(transformers=[
    ('numerical', numeric_transformer, numeric_features),
    ('categorical_low', categorical_transformer_ohe, categorical_features_ohe),
    ('categorical_high', categorical_transformer_loo, categorical_features_others)
])

# Preprocessor with HashingEncoder
preprocessor_hash = ColumnTransformer(transformers=[
    ('numerical', numeric_transformer, numeric_features),
    ('categorical_low', categorical_transformer_ohe, categorical_features_ohe),
    ('categorical_model', ce.HashingEncoder(n_components=3), 'model'),
    ('categorical_manufacter', ce.HashingEncoder(n_components=8), 'manufacter')
])

In [11]:
X = used_vehicles.loc[:, used_vehicles.columns != 'price']
y = used_vehicles['price']

In [12]:
X.shape

(99187, 9)

In [13]:
y.shape

(99187,)

### Modeling

As the goal of the exercise is to deploy a model, the RandomForestRegressor is going to be used for the sake of simplicity.

In [14]:
# Scale target
y = y.values.reshape(-1, 1)
price_scaler = StandardScaler().fit(y)
price_scaled = price_scaler.transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, price_scaled, test_size=0.2, random_state=0, shuffle=True)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(79349, 9) (79349, 1)
(19838, 9) (19838, 1)


#### Model with BinEcoder

In [ ]:
# Define full pipeline
regr = Pipeline(steps=[
    ('preprocessor', preprocessor_bin),
    ('regressor', Ridge())
])


regr

In [ ]:
# Fit the model
regr.fit(X_train, y_train)

# Show RMSE
pred_train = regr.predict(X_train)
print(f"MSE (training): {mean_squared_error(y_train, pred_train, squared=True)}")

pred_test = regr.predict(X_test)
print(f"MSE (test): {mean_squared_error(y_test, pred_test, squared=True)}")

In [ ]:
# Optimize model
param_grid = {
    'regressor__n_estimators': np.arange(50, 200, 25),
    'regressor__max_depth': np.arange(2, 20)
}

grid_search = GridSearchCV(regr, param_grid, cv=10, n_jobs=2)
grid_search

In [ ]:
grid_search.fit(X_train, y_train)

print("Best params:")
print(grid_search.best_params_)

In [ ]:
# Define full pipeline
regr = Pipeline(steps=[
    ('preprocessor', preprocessor_bin),
    ('regressor', RandomForestRegressor(verbose=1))
])

# Fit the model
regr.fit(X_train, y_train)

# Show RMSE
pred_train = regr.predict(X_train)
print(f"RMSE (training): {mean_squared_error(y_train, pred_train, squared=False)}")

pred_test = regr.predict(X_test)
print(f"RMSE (test): {mean_squared_error(y_test, pred_test, squared=False)}")

#### Model with LeaveOneOutEncoder

In [15]:
# Define full pipeline
regr = Pipeline(steps=[
    ('preprocessor', preprocessor_loo),
    ('regressor', Ridge())
])

regr

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['mileage', 'tax', 'mpg',
                                                   'engineSize',
                                                   'registrationAge']),
                                                 ('categorical_low',
                                                  OneHotEncoder(),
                                                  ['transmission', 'fuelType']),
                                                 ('categorical_high',
                                                  LeaveOneOutEncoder(),
                                                  ['manufacter', 'model'])])),
                ('regressor', Ridge())])

In [16]:
# Fit the model
regr.fit(X_train, y_train)

# Show RMSE
pred_train = regr.predict(X_train)
print(f"MSE (training): {mean_squared_error(y_train, pred_train, squared=True)}")

pred_test = regr.predict(X_test)
print(f"MSE (test): {mean_squared_error(y_test, pred_test, squared=True)}")

MSE (training): 0.16089076557964324
MSE (test): 0.15841752242783858


In [ ]:
# Optimize model
param_grid = {
    'regressor__n_estimators': np.arange(50, 200, 25),
    'regressor__max_depth': np.arange(2, 20)
}

grid_search = GridSearchCV(regr, param_grid, cv=10, n_jobs=2)
grid_search

In [ ]:
grid_search.fit(X_train, y_train)

print("Best params:")
print(grid_search.best_params_)

In [ ]:
# Define full pipeline
regr = Pipeline(steps=[
    ('preprocessor', preprocessor_bin),
    ('regressor', Ridge())
])

# Fit the model
regr.fit(X_train, y_train)

# Show RMSE
pred_train = regr.predict(X_train)
print(f"RMSE (training): {mean_squared_error(y_train, pred_train, squared=False)}")

pred_test = regr.predict(X_test)
print(f"RMSE (test): {mean_squared_error(y_test, pred_test, squared=False)}")

#### Model with HashingEncoder

In [ ]:
# Define full pipeline
regr = Pipeline(steps=[
    ('preprocessor', preprocessor_hash),
    ('regressor', Ridge())
])

regr

In [ ]:
# Fit the model
regr.fit(X_train, y_train)

# Show RMSE
pred_train = regr.predict(X_train)
print(f"MSE (training): {mean_squared_error(y_train, pred_train, squared=True)}")

pred_test = regr.predict(X_test)
print(f"MSE (test): {mean_squared_error(y_test, pred_test, squared=True)}")

In [ ]:
# Optimize model
param_grid = {
    'preprocessor__categorical_model__n_components': np.arange(2, 5),
    'preprocessor__categorical_manufacter__n_components': np.arange(2, 10),
    'regressor__n_estimators': np.arange(50, 200, 25),
    'regressor__max_depth': np.arange(2, 20)
}

grid_search = GridSearchCV(regr, param_grid, cv=10, n_jobs=2)
grid_search

In [ ]:
grid_search.fit(X_train, y_train)

print("Best params:")
print(grid_search.best_params_)

In [ ]:
# Define full pipeline
regr = Pipeline(steps=[
    ('preprocessor', preprocessor_bin),
    ('regressor', Ridge())
])

# Fit the model
regr.fit(X_train, y_train)

# Show RMSE
pred_train = regr.predict(X_train)
print(f"RMSE (training): {mean_squared_error(y_train, pred_train, squared=False)}")

pred_test = regr.predict(X_test)
print(f"RMSE (test): {mean_squared_error(y_test, pred_test, squared=False)}")

### Export model

In [ ]:
with open('model.zlib', 'wb') as file:
    joblib.dump(regr, file)

In [17]:
with open('data/target_scaler.zlib', 'wb') as file:
    joblib.dump(price_scaler, file)

### Test model import

In [ ]:
regr_disk = joblib.load('model.zlib')

In [ ]:
result = regr_disk.predict(X_test[:10])
print(f"Unscaled prediction {result.tolist()[0]}")
transformed_prediction = price_scaler.inverse_transform(result.reshape(-1,1)) # Note the price_scaler is the target scaler
print(f"transformed_prediction {transformed_prediction.tolist()[0]}")